In [1]:
import numpy as np

In [2]:
# Set the seed for the numpy random number generator.
# This ensures that we get reproducible results
np.random.seed(17335)

## Generating a sequence of states from a 1st order discrete Markov process SIR model

We’re going to start from the susceptible state, sample the next state using the probabilities from the first row of the SIR rate matrix. We’ll repeat a further 9 times, each time using the probability distribution from the row of the rate matrix corresponding to the current state. At the end we will have a sequence of 11 states. This will be an example trajectory from our SIR 1st order Markov process and could represent the trajectory of an individual person.

In [3]:
### Define our states 
# Our states are "Susceptible" = index 0, 
# "Infected" = index 1
# "Recovered" = index 2
state_map = {0:'S', 1:'I', 2:'R'}

### Define our rate matrix
rate_matrix = np.array([[0.9, 0.1, 0.0],[0.0, 0.8, 0.2],[0.05, 0.0, 0.95]])

### We'll start in the susceptible state and then sample 
### our next state and repeat for 10 iterations
n_iter=10

# Set initial state and its label
current_state = 0
states_sequence = [state_map[current_state]]

# Do 10 iterations
for i in range(n_iter):
    # Get transition probabilities when starting from 
    # our current state
    next_state_probs = rate_matrix[current_state,:]
    
    # Use numpy function to sample an integer from [0,1,2]
    # with the specified transition probabilities 
    next_state = np.random.choice(3, p=next_state_probs)
    
    # Update our current state to this new state
    # and get its label
    current_state = next_state
    states_sequence.extend(state_map[current_state])

In [4]:
# Let's look at the sequence of state labels for the 
# sequence of states we have generated
print(states_sequence)

['S', 'S', 'S', 'S', 'S', 'I', 'R', 'R', 'R', 'R', 'R']


## Calculation of the stable distribution and the limiting distribution

Any stable distribution of the 1st order Markov process is given by a right-eigenvector of the transpose of the transition rate matrix corresponding to eigenvalue 1. We can use the numpy.linalg.eigen function, which computes the right-eigenvectors and eigenvalues of a square matrix.

In [5]:
# Compute the right-eigenvectors and eigenvalues of our rate matrix
rate_eigen = np.linalg.eig(np.transpose(rate_matrix))

We can then look at the eigenvalues

In [6]:
# The eigenvalues are held in the first element of the tuple
rate_eigen[0]

array([0.825+0.06614378j, 0.825-0.06614378j, 1.   +0.j        ])

The last eigenvalue is 1(its imaginary part is zero). So we can use the last eigenvector as the stable distribution. It will normalized to unit length, but we need its elements to sum to 1. But remember from Chapter 3 that any multiple of an eigenvector is still an eigenvector with the same eigenvalue, so we can just re-scale it to sum to 1.

In [7]:
# Extract the last eigenvector (we can drop the zero imaginary part)
pi_stable = np.real(rate_eigen[1][:,2])

# Rescale the vector so its elements sumto 1
pi_stable /=np.sum(pi_stable)

Let's take a look at the stable distribution

In [8]:
# Print the stable distribution
pi_stable

array([0.28571429, 0.14285714, 0.57142857])

Now we'll compute the limiting distribution. We'll do this by starting from a definite initial state, i.e. a distribution which has only one non-zero value. We'll then apply the transition rate matrix a large (1000) number of times to get an approximation to the limiting distribution.

In [9]:
# Set the initial state distribution. We'll 
# set it to a distribution representing 100% of people 
# being in the susceptible state
current_distribution = np.array([1.0, 0.0, 0.0])

n_iter = 100
for i in range(n_iter):
    # Get the state distribution at the next timepoint by 
    # multiplying by the transition rate matrix
    next_distribution = np.matmul(current_distribution, rate_matrix)
    current_distribution = next_distribution
    
current_distribution

array([0.28571429, 0.14285715, 0.57142856])

We'll repeat the above exercise, but starting from a different initial state,to show that we still end up with the same limiting distribution. We'll create a function to run the code above because we'll call it several time.

In [10]:
def estimate_limiting_distribution(initial_distribution, rate_matrix, n_iter):
    '''
    Function to iterate the evolution of the state probability distribution
    
    :param initial distribution: The initial state distribution from which we start the evolution.
    :type initial distribution: An 1D numpy array of N elements.
    
    :param rate_matrix: The transition probability matrix.
    :type rate_matrix: A 2D numpy square N x N array.
    
    :param n_iter: The number of times we will iterate the evolution of the state distribution.
    :type n_iter: int
    
    :return: The state distribution after n_iter iterations of the evolution equation
               starting from initial_distribution.
    :rtype: An 1D numpy array of N elements.
    '''
    
    # Set the current distribution equal to the initial distribution
    current_distribution = initial_distribution
    
    # Perform n_iter iterations of the evolution equation,
    # updating the current distribution as we go along.
    for i in range(n_iter):
        # Get the state distribution at the next timepoint by 
        # multiplying by the transition rate matrix
        next_distribution = np.matmul(current_distribution, rate_matrix)
        current_distribution = next_distribution    
        
    return current_distribution

In [11]:
# Estimate the limiting distribution starting from an initial distribution that is 
# 100% infected state
limiting_distribution_estimate1 = estimate_limiting_distribution(initial_distribution=np.array([0.0, 1.0, 0.0]),
                                                                 rate_matrix=rate_matrix,
                                                                 n_iter=100)

# Look at the estimate of the limiting distribution
print("Estimate of the limiting distribution starting from [0, 1, 0]")
limiting_distribution_estimate1 

Estimate of the limiting distribution starting from [0, 1, 0]


array([0.28571428, 0.14285714, 0.57142858])

In [12]:
# Estimate the limiting distribution starting from an initial distribution that is uniform
limiting_distribution_estimate2 = estimate_limiting_distribution(initial_distribution=np.array([1.0/3.0, 1.0/3.0, 1.0/3.0]),
                                                                 rate_matrix=rate_matrix,
                                                                 n_iter=100)

# Look at the estimate of the limiting distribution
print("Estimate of the limiting distribution starting from [1/3, 1/3, 1/3]")
limiting_distribution_estimate2 

Estimate of the limiting distribution starting from [1/3, 1/3, 1/3]


array([0.28571428, 0.14285714, 0.57142857])

We can see that the two estimates of the limiting distribution are the same (to within the precision of the numerical calculations).

As a final code example on 1st order discrete Markov processes, we'll confirm the limiting state distribution by that the state by generating lots of long trajectories and looking at the frequency of the final states reached. This code can take several minutes to run.

In [13]:
# Set the number of trajectories we're going to generate
n_trajectories = 10000

# Set the length of the trajectories
n_iter=1000

# Initialize an array to hold the counts of final states
final_state_distribution =np.zeros(3)

# Loop over the trajectories
for i in range(n_trajectories):
    if i%500==0:
        print("Running trajectory " + str(i) + " out of " + str(n_trajectories))
    
    # Set initial state and its label
    current_state = 0

    # Do a large number of iterations
    # and record the final state
    for j in range(n_iter):
        # Get transition probabilities when starting from 
        # our current state
        next_state_probs = rate_matrix[current_state,:]
    
        # Use numpy function to sample an integer from [0,1,2]
        # with the specified transition probabilities 
        next_state = np.random.choice(3, size=1, p=next_state_probs)[0]
    
        # Update our current state to this new state
        current_state = next_state
    
    final_state_distribution[current_state] += 1.0

# Convert the state frequencies into proportions    
final_state_distribution /= float(n_trajectories)        

Running trajectory 0 out of 10000
Running trajectory 500 out of 10000
Running trajectory 1000 out of 10000
Running trajectory 1500 out of 10000
Running trajectory 2000 out of 10000
Running trajectory 2500 out of 10000
Running trajectory 3000 out of 10000
Running trajectory 3500 out of 10000
Running trajectory 4000 out of 10000
Running trajectory 4500 out of 10000
Running trajectory 5000 out of 10000
Running trajectory 5500 out of 10000
Running trajectory 6000 out of 10000
Running trajectory 6500 out of 10000
Running trajectory 7000 out of 10000
Running trajectory 7500 out of 10000
Running trajectory 8000 out of 10000
Running trajectory 8500 out of 10000
Running trajectory 9000 out of 10000
Running trajectory 9500 out of 10000


Let's look at the final state distribution to compare it to the estimate of the limiting distribution and the stable distribution.

In [14]:
# Print the estimate final state distribution
final_state_distribution

array([0.2835, 0.1459, 0.5706])